# Group operations - split-apply-combine
- grouping key can take many forms:
 - index or columns
 - keys do not have to be of same type
 - a list/array same length as grouping axis
 - a dictionary/Series corresponding to values on grouping axis
 - a function to be invoked on the axis or individual labels on the index

In [1]:
import pandas as pd
import numpy as np

In [2]:

np.random.seed(42)
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
"key2" : pd.Series([1, 2, 1, 2, 1, np.nan, 1], dtype="Int64"),
"data1" : np.random.standard_normal(7),
"data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,0.496714,0.767435
1,a,2,-0.138264,-0.469474
2,None,1,0.647689,0.542560
3,b,2,1.523030,-0.463418
4,b,1,-0.234153,-0.465730
5,a,<NA>,-0.234137,0.241962
6,None,1,1.579213,-1.913280


In [3]:
df["key2"].dtype

Int64Dtype()

In [4]:
df["key1"].isna()

0    False
1    False
2     True
3    False
4    False
5    False
6     True
Name: key1, dtype: bool

In [5]:
# df = df.set_index(keys="key1")
# df.index.name = "key1"
df.groupby(by="key1", dropna=False).mean()

,key2,data1,data2
key1,,,
a,1.5,0.041438,0.179974
b,1.5,0.644438,-0.464574
NaN,1.0,1.113451,-0.685360


In [6]:
grouped = df["data1"].groupby(df["key1"])
grouped.mean()

key1
a    0.041438
b    0.644438
Name: data1, dtype: float64

In [7]:
mean_df = df.groupby(by=["key1", "key2"]).mean()

In [8]:
mean_df

data1     data2
key1 key2                    
a    1     0.496714  0.767435
     2    -0.138264 -0.469474
b    1    -0.234153 -0.465730
     2     1.523030 -0.463418

In [9]:
mean_df = mean_df.unstack()

In [10]:
mean_df.columns.nlevels

2

In [11]:
mean_df

data1               data2          
key2         1         2         1         2
key1                                        
a     0.496714 -0.138264  0.767435 -0.469474
b    -0.234153  1.523030 -0.465730 -0.463418

In [12]:
mean_df.reset_index()

key1     data1               data2          
key2              1         2         1         2
0       a  0.496714 -0.138264  0.767435 -0.469474
1       b -0.234153  1.523030 -0.465730 -0.463418

In [13]:
mean_df.columns.names

FrozenList([None, 'key2'])

In [14]:
mean_df.columns.names = ["data", "key2"]
mean_df

data     data1               data2          
key2         1         2         1         2
key1                                        
a     0.496714 -0.138264  0.767435 -0.469474
b    -0.234153  1.523030 -0.465730 -0.463418

In [15]:
mean_df.swaplevel(i="data", j="key2", axis=1)

key2,1,2,1,2
data,data1,data1,data2,data2
key1,,,,
a,0.496714,-0.138264,0.767435,-0.469474
b,-0.234153,1.523030,-0.465730,-0.463418


In [16]:
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

- `count` computes number of non-null values in group

In [17]:
df.groupby(["key1"], dropna=False).count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2
NaN,2,2,2


In [20]:
for key1, group in df.groupby("key1"):
    print(key1)
    print(group)

a
  key1  key2     data1     data2
0    a     1  0.496714  0.767435
1    a     2 -0.138264 -0.469474
5    a  <NA> -0.234137  0.241962
b
  key1  key2     data1     data2
3    b     2  1.523030 -0.463418
4    b     1 -0.234153 -0.465730


In [21]:
for (key1, key2), group in df.groupby(["key1", "key2"]):
    print(key1, key2)
    print(group)

a 1
  key1  key2     data1     data2
0    a     1  0.496714  0.767435
a 2
  key1  key2     data1     data2
1    a     2 -0.138264 -0.469474
b 1
  key1  key2     data1    data2
4    b     1 -0.234153 -0.46573
b 2
  key1  key2    data1     data2
3    b     2  1.52303 -0.463418


In [22]:
pieces = {name: group for name, group in df.groupby(["key1", "key2"])}
pieces

{('a',
  1):   key1  key2     data1     data2
 0    a     1  0.496714  0.767435,
 ('a',
  2):   key1  key2     data1     data2
 1    a     2 -0.138264 -0.469474,
 ('b',
  1):   key1  key2     data1    data2
 4    b     1 -0.234153 -0.46573,
 ('b',
  2):   key1  key2    data1     data2
 3    b     2  1.52303 -0.463418}

on `columns` axis
- essentially splits the data vertically into groups of unique columns

following example we create two groups - `data` group containing data columns and `key` group containing key columns


In [23]:
grouped_cols = df.groupby({"key1": "key", "key2": "key", "data1": "data", "data2": "data"}, axis=1)
for name, group in grouped_cols:
    print(name)
    print(group)

data
      data1     data2
0  0.496714  0.767435
1 -0.138264 -0.469474
2  0.647689  0.542560
3  1.523030 -0.463418
4 -0.234153 -0.465730
5 -0.234137  0.241962
6  1.579213 -1.913280
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


In [25]:
s_grouped = df.groupby(["key1", "key2"])["data1"]

In [26]:
print(*s_grouped)

(('a', 1), 0    0.496714
Name: data1, dtype: float64) (('a', 2), 1   -0.138264
Name: data1, dtype: float64) (('b', 1), 4   -0.234153
Name: data1, dtype: float64) (('b', 2), 3    1.52303
Name: data1, dtype: float64)


 by `dictionary` and `Series`

In [27]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
columns=["a", "b", "c", "d", "e"],
index=["Joe", "Steve", "Wanda", "Jill", "Trey"])

people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
 
people

,a,b,c,d,e
Joe,-1.724918,-0.562288,-1.012831,0.314247,-0.908024
Steve,-1.412304,1.465649,-0.225776,0.067528,-1.424748
Wanda,-0.544383,NaN,NaN,0.375698,-0.600639
Jill,-0.291694,-0.601707,1.852278,-0.013497,-1.057711
Trey,0.822545,-1.220844,0.208864,-1.959670,-1.328186


In [28]:
grp_mapping = {"a": "red",
"b": "red", 
"c": "orange",
"d": "orange",
"e": "red",
"f": "green" } # unused group mappings are ok
by_col = people.groupby(grp_mapping, axis=1, dropna=False)
by_col.mean()

,orange,red
Joe,-0.349292,-1.065076
Steve,-0.079124,-0.457134
Wanda,0.375698,-0.572511
Jill,0.919390,-0.650370
Trey,-0.875403,-0.575495


In [31]:
map_series = pd.Series(grp_mapping)
map_series

by_col = people.groupby(grp_mapping, axis=1, dropna=False)
by_col.mean()
#by_col.count()

,orange,red
Joe,-0.349292,-1.065076
Steve,-0.079124,-0.457134
Wanda,0.375698,-0.572511
Jill,0.919390,-0.650370
Trey,-0.875403,-0.575495


- by `functions`

In [32]:
# group over all people with same length of names
people.groupby(len).count() # computes len on each index value and return value used as index

,a,b,c,d,e
3,1,1,1,1,1
4,2,2,2,2,2
5,2,1,1,2,2


- mix arrays with functions

In [34]:
key_list = ["one", "one", "one", "two", "two"] # len of key_list equal to number of rows in people
people.groupby([len, key_list]).count() # first group by len and then by key_list

,,a,b,c,d,e
3,one,1,1,1,1,1
4,two,2,2,2,2,2
5,one,2,1,1,2,2


- groupby `index levels`

In [37]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
    [1, 3, 5, 1, 3]],
    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns = columns)
hier_df


cty          US                            JP          
tenor         1         3         5         1         3
0     -1.987569 -0.219672  0.357113  1.477894 -0.518270
1     -0.808494 -0.501757  0.915402  0.328751 -0.529760
2      0.513267  0.097078  0.968645 -0.702053 -0.327662
3     -0.392108 -1.463515  0.296120  0.261055  0.005113

In [38]:
# level=1 is grouping by tenor
hier_df.groupby(level=1, axis="columns").mean()

tenor,1,3,5
0,-0.254837,-0.368971,0.357113
1,-0.239871,-0.515759,0.915402
2,-0.094393,-0.115292,0.968645
3,-0.065526,-0.729201,0.296120


# Data Aggregation

In [39]:
df

,key1,key2,data1,data2
0,a,1,0.496714,0.767435
1,a,2,-0.138264,-0.469474
2,None,1,0.647689,0.542560
3,b,2,1.523030,-0.463418
4,b,1,-0.234153,-0.465730
5,a,<NA>,-0.234137,0.241962
6,None,1,1.579213,-1.913280


In [40]:
grouped = df.groupby("key1")

def max_plus1(arr):
    return arr.max() + 1

grouped.apply(max_plus1)

,key2,data1,data2
key1,,,
a,3.0,1.496714,1.767435
b,3.0,2.523030,0.536582


In [44]:
grouped.agg({"key2": "count", "data1": max_plus1})

,key2,data1
key1,,
a,2,1.496714
b,2,2.523030


In [45]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.041438  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.644438  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.179225  0.496714   3.0  0.179974  0.620780 -0.469474 -0.113756   
b     1.083734  1.523030   2.0 -0.464574  0.001635 -0.465730 -0.465152   

                                    
           50%       75%       max  
key1                                
a     0.241962  0.504699  0.767435  
b    -0.464574 -0.463996 -0.463418  

[2 rows x 24 columns]

In [46]:
grouped.mean()

,key2,data1,data2
key1,,,
a,1.5,0.041438,0.179974
b,1.5,0.644438,-0.464574


# Column-wise and Multiple function application

In [47]:
tips = pd.read_csv("examples/tips.csv")
tips["tip_pct"] = tips["tip"]/ tips["total_bill"]
tips

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [48]:
tips_grp = tips.groupby(["smoker", "day"])
tips_grp.describe()

total_bill                                                        \
                 count       mean        std    min     25%     50%      75%   
smoker day                                                                     
No     Fri         4.0  18.420000   5.059282  12.46  15.100  19.235  22.5550   
       Sat        45.0  19.661778   8.939181   7.25  14.730  17.820  20.6500   
       Sun        57.0  20.506667   8.130189   8.77  14.780  18.430  25.0000   
       Thur       45.0  17.113111   7.721728   7.51  11.690  15.950  20.2700   
Yes    Fri        15.0  16.813333   9.086388   5.75  11.690  13.420  18.6650   
       Sat        42.0  21.276667  10.069138   3.07  13.405  20.390  26.7925   
       Sun        19.0  24.120000  10.442511   7.25  17.165  23.100  32.3750   
       Thur       17.0  19.190588   8.355149  10.34  13.510  16.470  19.8100   

                     tip            ...  size      tip_pct            \
               max count      mean  ...   75%  max   count      mean   
smoker day                          ...                                
No     Fri   22.75   4.0  2.812500  ...  2.25  3.0     4.0  0.151650   
       Sat   48.33  45.0  3.102889  ...  3.00  4.0    45.0  0.158048   
       Sun   48.17  57.0  3.167895  ...  4.00  6.0    57.0  0.160113   
       Thur  41.19  45.0  2.673778  ...  2.00  6.0    45.0  0.160298   
Yes    Fri   40.17  15.0  2.714000  ...  2.00  4.0    15.0  0.174783   
       Sat   50.81  42.0  2.875476  ...  3.00  5.0    42.0  0.147906   
       Sun   45.35  19.0  3.516842  ...  3.00  5.0    19.0  0.187250   
       Thur  43.11  17.0  3.030000  ...  2.00  4.0    17.0  0.163863   

                                                                         
                  std       min       25%       50%       75%       max  
smoker day                                                               
No     Fri   0.028123  0.120385  0.137239  0.149241  0.163652  0.187735  
       Sat   0.039767  0.056797  0.136240  0.150152  0.183915  0.291990  
       Sun   0.042347  0.059447  0.139780  0.161665  0.185185  0.252672  
       Thur  0.038774  0.072961  0.137741  0.153492  0.184843  0.266312  
Yes    Fri   0.051293  0.103555  0.133739  0.173913  0.209240  0.263480  
       Sat   0.061375  0.035638  0.091797  0.153624  0.190502  0.325733  
       Sun   0.154134  0.065660  0.097723  0.138122  0.215325  0.710345  
       Thur  0.039389  0.090014  0.148038  0.153846  0.194837  0.241255  

[8 rows x 32 columns]

In [50]:
tips_grp.agg(["mean", "std", max_plus1]) # multiple aggregate functions on all columns

/var/folders/nx/j3jrdkw97q1gxc0nkyjz0sg80000gn/T/ipykernel_1372/502261771.py:1: FutureWarning: ['time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  tips_grp.agg(["mean", "std", max_plus1]) # multiple aggregate functions on all columns


total_bill                            tip                      \
                  mean        std max_plus1      mean       std max_plus1   
smoker day                                                                  
No     Fri   18.420000   5.059282     23.75  2.812500  0.898494      4.50   
       Sat   19.661778   8.939181     49.33  3.102889  1.642088     10.00   
       Sun   20.506667   8.130189     49.17  3.167895  1.224785      7.00   
       Thur  17.113111   7.721728     42.19  2.673778  1.282964      7.70   
Yes    Fri   16.813333   9.086388     41.17  2.714000  1.077668      5.73   
       Sat   21.276667  10.069138     51.81  2.875476  1.630580     11.00   
       Sun   24.120000  10.442511     46.35  3.516842  1.261151      7.50   
       Thur  19.190588   8.355149     44.11  3.030000  1.113491      6.00   

                 size                       tip_pct                      
                 mean       std max_plus1      mean       std max_plus1  
smoker day                                                               
No     Fri   2.250000  0.500000         4  0.151650  0.028123  1.187735  
       Sat   2.555556  0.784960         5  0.158048  0.039767  1.291990  
       Sun   2.929825  1.032674         7  0.160113  0.042347  1.252672  
       Thur  2.488889  1.179796         7  0.160298  0.038774  1.266312  
Yes    Fri   2.066667  0.593617         5  0.174783  0.051293  1.263480  
       Sat   2.476190  0.862161         6  0.147906  0.061375  1.325733  
       Sun   2.578947  0.901591         6  0.187250  0.154134  1.710345  
       Thur  2.352941  0.701888         5  0.163863  0.039389  1.241255

In [51]:
tips_grp.agg([("average", "mean"), ("std-dev", lambda x: x.std())]) # customizing column names

/var/folders/nx/j3jrdkw97q1gxc0nkyjz0sg80000gn/T/ipykernel_1372/1390994588.py:1: FutureWarning: ['time'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  tips_grp.agg([("average", "mean"), ("std-dev", lambda x: x.std())])


total_bill                  tip                size            \
               average    std-dev   average   std-dev   average   std-dev   
smoker day                                                                  
No     Fri   18.420000   5.059282  2.812500  0.898494  2.250000  0.500000   
       Sat   19.661778   8.939181  3.102889  1.642088  2.555556  0.784960   
       Sun   20.506667   8.130189  3.167895  1.224785  2.929825  1.032674   
       Thur  17.113111   7.721728  2.673778  1.282964  2.488889  1.179796   
Yes    Fri   16.813333   9.086388  2.714000  1.077668  2.066667  0.593617   
       Sat   21.276667  10.069138  2.875476  1.630580  2.476190  0.862161   
       Sun   24.120000  10.442511  3.516842  1.261151  2.578947  0.901591   
       Thur  19.190588   8.355149  3.030000  1.113491  2.352941  0.701888   

              tip_pct            
              average   std-dev  
smoker day                       
No     Fri   0.151650  0.028123  
       Sat   0.158048  0.039767  
       Sun   0.160113  0.042347  
       Thur  0.160298  0.038774  
Yes    Fri   0.174783  0.051293  
       Sat   0.147906  0.061375  
       Sun   0.187250  0.154134  
       Thur  0.163863  0.039389

- customs function for each column

In [53]:
from functools import partial

In [ ]:
np.unique()

In [54]:
tips_grp.agg({"total_bill": [("bill_max", "max"), "mean"], "time": ["count", partial(np.unique, return_counts=True)], "tip": ["mean", "min"]})

total_bill             time                                   tip  \
              bill_max       mean count                      unique      mean   
smoker day                                                                      
No     Fri       22.75  18.420000     4   ([Dinner, Lunch], [3, 1])  2.812500   
       Sat       48.33  19.661778    45            ([Dinner], [45])  3.102889   
       Sun       48.17  20.506667    57            ([Dinner], [57])  3.167895   
       Thur      41.19  17.113111    45  ([Dinner, Lunch], [1, 44])  2.673778   
Yes    Fri       40.17  16.813333    15   ([Dinner, Lunch], [9, 6])  2.714000   
       Sat       50.81  21.276667    42            ([Dinner], [42])  2.875476   
       Sun       45.35  24.120000    19            ([Dinner], [19])  3.516842   
       Thur      43.11  19.190588    17             ([Lunch], [17])  3.030000   

                   
              min  
smoker day         
No     Fri   1.50  
       Sat   1.00  
       Sun   1.01  
       Thur  1.25  
Yes    Fri   1.00  
       Sat   1.00  
       Sun   1.50  
       Thur  2.00

In [55]:
# disable index
tips.groupby(["day", "smoker"], group_keys=False)[["total_bill", "tip"]].agg(["mean", "std"]) 


total_bill                  tip          
                  mean        std      mean       std
day  smoker                                          
Fri  No      18.420000   5.059282  2.812500  0.898494
     Yes     16.813333   9.086388  2.714000  1.077668
Sat  No      19.661778   8.939181  3.102889  1.642088
     Yes     21.276667  10.069138  2.875476  1.630580
Sun  No      20.506667   8.130189  3.167895  1.224785
     Yes     24.120000  10.442511  3.516842  1.261151
Thur No      17.113111   7.721728  2.673778  1.282964
     Yes     19.190588   8.355149  3.030000  1.113491

# Split-Apply-Combine

`apply` maps a function to each group whereas `agg` maps a function to each column in the group

In [56]:
def top5(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

tips.groupby("smoker").apply(top5)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     232       11.61  3.39     No   Sat  Dinner     2  0.291990
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       88        24.71  5.85     No  Thur   Lunch     2  0.236746
Yes    172        7.25  5.15    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00    Yes   Sat  Dinner     2  0.279525

In [59]:
top5(tips, n=2, column="total_bill")

,total_bill,tip,smoker,day,time,size,tip_pct
170,50.81,10.0,Yes,Sat,Dinner,3,0.196812
212,48.33,9.0,No,Sat,Dinner,4,0.186220


In [57]:
tips.groupby("smoker").apply(top5, n=2, column="total_bill")

total_bill    tip smoker  day    time  size   tip_pct
smoker                                                           
No     212       48.33   9.00     No  Sat  Dinner     4  0.186220
       59        48.27   6.73     No  Sat  Dinner     4  0.139424
Yes    170       50.81  10.00    Yes  Sat  Dinner     3  0.196812
       182       45.35   3.50    Yes  Sun  Dinner     3  0.077178

In [62]:
# suppress indexing on group key
tips.groupby("smoker", group_keys=False).apply(top5, n=2, column="total_bill")

,total_bill,tip,smoker,day,time,size,tip_pct
212,48.33,9.00,No,Sat,Dinner,4,0.186220
59,48.27,6.73,No,Sat,Dinner,4,0.139424
170,50.81,10.00,Yes,Sat,Dinner,3,0.196812
182,45.35,3.50,Yes,Sun,Dinner,3,0.077178


example: quantile & bucket analysis `pd.cut`

In [64]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                     "data2": np.random.standard_normal(1000)})
frame.head()                     
                     

,data1,data2
0,-0.234587,0.346488
1,-1.415371,0.998010
2,-0.420645,-2.896255
3,-0.342715,2.088375
4,-0.802277,-0.139590


In [65]:
quartiles = pd.cut(frame["data1"], 4) # slice data into bins
quartiles.head(10)

0    (-1.468, 0.306]
1    (-1.468, 0.306]
2    (-1.468, 0.306]
3    (-1.468, 0.306]
4    (-1.468, 0.306]
5    (-1.468, 0.306]
6     (0.306, 2.079]
7     (0.306, 2.079]
8    (-1.468, 0.306]
9    (-1.468, 0.306]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.248, -1.468] < (-1.468, 0.306] < (0.306, 2.079] < (2.079, 3.853]]

In [68]:
grouped = frame.groupby(quartiles)
grouped.agg(["min", "max", "count", "mean"])

data1                               data2            \
                       min       max count      mean       min       max   
data1                                                                      
(-3.248, -1.468] -3.241267 -1.478586    59 -1.853405 -1.768439  2.157308   
(-1.468, 0.306]  -1.464375  0.301547   552 -0.433880 -2.940389  3.193108   
(0.306, 2.079]    0.307300  2.076748   367  0.927744 -3.019512  3.137749   
(2.079, 3.853]    2.092387  3.852731    22  2.432890 -1.993736  1.749577   

                                  
                 count      mean  
data1                             
(-3.248, -1.468]    59  0.046201  
(-1.468, 0.306]    552  0.065007  
(0.306, 2.079]     367  0.023192  
(2.079, 3.853]      22 -0.178836

In [69]:
def get_stats(group):
    return pd.DataFrame({
        "min": group.min(),
        "mean": group.mean(),
        "count": group.count()
    })
grouped.apply(get_stats)

min      mean  count
data1                                            
(-3.248, -1.468] data1 -3.241267 -1.853405     59
                 data2 -1.768439  0.046201     59
(-1.468, 0.306]  data1 -1.464375 -0.433880    552
                 data2 -2.940389  0.065007    552
(0.306, 2.079]   data1  0.307300  0.927744    367
                 data2 -3.019512  0.023192    367
(2.079, 3.853]   data1  2.092387  2.432890     22
                 data2 -1.993736 -0.178836     22

`pd.qcut` to compute equal-size buckets based on sample quantiles

In [70]:
quartiles_samp = pd.qcut(frame["data1"], 4)
quartiles_samp.head()

0    (-0.623, 0.0477]
1    (-3.242, -0.623]
2    (-0.623, 0.0477]
3    (-0.623, 0.0477]
4    (-3.242, -0.623]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.242, -0.623] < (-0.623, 0.0477] < (0.0477, 0.676] < (0.676, 3.853]]

In [72]:
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min      mean  count
data1                                            
(-3.242, -0.623] data1 -3.241267 -1.188417    250
                 data2 -2.940389  0.096543    250
(-0.623, 0.0477] data1 -0.622700 -0.284734    250
                 data2 -2.921350  0.049074    250
(0.0477, 0.676]  data1  0.048085  0.352752    250
                 data2 -2.153390 -0.033442    250
(0.676, 3.853]   data1  0.679598  1.301010    250
                 data2 -3.019512  0.060572    250

example: filling missing data with group specific values
- instead of dropping values , fill them 

In [73]:
s = pd.Series(np.random.standard_normal(16))
s[::2] = np.nan
s

0          NaN
1     0.376410
2          NaN
3    -0.869663
4          NaN
5    -1.189412
6          NaN
7    -0.900621
8          NaN
9    -0.328618
10         NaN
11   -0.544114
12         NaN
13    0.040919
14         NaN
15    0.740824
dtype: float64

In [74]:
s.fillna(s.mean())

0    -0.334284
1     0.376410
2    -0.334284
3    -0.869663
4    -0.334284
5    -1.189412
6    -0.334284
7    -0.900621
8    -0.334284
9    -0.328618
10   -0.334284
11   -0.544114
12   -0.334284
13    0.040919
14   -0.334284
15    0.740824
dtype: float64

In [75]:
frame

,data1,data2
0,-0.234587,0.346488
1,-1.415371,0.998010
2,-0.420645,-2.896255
3,-0.342715,2.088375
4,-0.802277,-0.139590
...,...,...
995,0.732640,1.039112
996,-0.080717,-0.075764
997,0.078635,0.670481
998,-1.998201,-1.071857


In [77]:
frame["data2"][::2] = np.nan
frame

,data1,data2
0,-0.234587,NaN
1,-1.415371,0.998010
2,-0.420645,NaN
3,-0.342715,2.088375
4,-0.802277,NaN
...,...,...
995,0.732640,1.039112
996,-0.080717,NaN
997,0.078635,0.670481
998,-1.998201,NaN


In [79]:
grouped = frame.groupby(quartiles_samp)

In [83]:
grouped

In [88]:
def fillna_grps(group):
    return group.fillna(group.mean())

In [89]:
filled_frame = grouped.apply(fillna_grps)

In [90]:
filled_frame

,data1,data2
0,-0.234587,0.072471
1,-1.415371,0.998010
2,-0.420645,0.072471
3,-0.342715,2.088375
4,-0.802277,0.130410
...,...,...
995,0.732640,1.039112
996,-0.080717,0.072471
997,0.078635,0.670481
998,-1.998201,0.130410


In [84]:
grouped.apply(get_stats)

min      mean  count
data1                                            
(-3.242, -0.623] data1 -3.241267 -1.188417    250
                 data2 -2.703232  0.130410    121
(-0.623, 0.0477] data1 -0.622700 -0.284734    250
                 data2 -2.530288  0.072471    126
(0.0477, 0.676]  data1  0.048085  0.352752    250
                 data2 -2.151815 -0.119228    131
(0.676, 3.853]   data1  0.679598  1.301010    250
                 data2 -3.019512 -0.006981    122

In [91]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]

group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]

data = pd.Series(np.random.standard_normal(8), index=states)

In [94]:
data[["Vermont", "Nevada", "Florida", "Idaho"]] = np.nan
print(data)

data.groupby(group_key).apply(fillna_grps)

Ohio         -0.513214
New York     -0.228600
Vermont            NaN
Florida            NaN
Oregon       -0.191028
Nevada             NaN
California    0.784604
Idaho              NaN
dtype: float64


Ohio         -0.513214
New York     -0.228600
Vermont      -0.370907
Florida      -0.370907
Oregon       -0.191028
Nevada        0.296788
California    0.784604
Idaho         0.296788
dtype: float64

In [95]:
(-0.513214-0.228600)/2

-0.370907

In [96]:
(-0.191028+0.784604)/2

0.296788

In [98]:
fillvalues = {"East": 0.4, "West": 0.5}
def fill_bygroupnames(group):
    return group.fillna(fillvalues[group.name])
data.groupby(group_key).apply(fill_bygroupnames)

Ohio         -0.513214
New York     -0.228600
Vermont       0.400000
Florida       0.400000
Oregon       -0.191028
Nevada        0.500000
California    0.784604
Idaho         0.500000
dtype: float64

example: random sampling and permutation
use `sample` method for the series

In [102]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards, name="values")
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
Name: values, dtype: int64

In [104]:
def draw_hand(deck, n=5):
    return deck.sample(n)
draw_hand(deck)

2H     2
5D     5
AS     1
5S     5
JS    10
Name: values, dtype: int64

In [106]:
# draw two random cards from each suit, suit is last name of each card
def get_suit(card):
    return card[-1]

deck.groupby(get_suit).apply(draw_hand, n=2)


C  6C      6
   10C    10
D  4D      4
   5D      5
H  2H      2
   6H      6
S  3S      3
   QS     10
Name: values, dtype: int64

In [107]:

deck.groupby(get_suit, group_keys=False).apply(draw_hand, n=2)

7C      7
JC     10
AD      1
3D      3
10H    10
JH     10
QS     10
5S      5
Name: values, dtype: int64

example: group weighted average and correlation

In [118]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
               "b", "b", "b", "b"],
  "data": np.random.standard_normal(8),
  "data1": np.random.standard_normal(8),
  "weights": np.random.uniform(size=8)})

In [129]:
def weighted_average(df, weight_column):
    average = {}
    weight_sum = df[weight_column].sum()
    for column in df.drop([weight_column], axis=1).columns:
        # df[column] *= df[weight_column]
        # average[column] = df[column].sum() / weight_sum
        average[column] = np.average(df[column], weights=df[weight_column])
    return pd.Series(average)

weighted_average(df.drop(["category"], axis=1), "weights")


data     0.008820
data1    0.138883
dtype: float64

In [130]:
df.groupby("category").apply(weighted_average, weight_column="weights")

,data,data1
category,,
a,0.002384,0.448967
b,0.013452,-0.084307


In [152]:
close_px = pd.read_csv("examples/stock_px_2.csv", parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [153]:
close_px.head()

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93


compute yearly correlations of returns with SPX

In [154]:
returns = close_px.pct_change().dropna()
returns.head()

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386


In [155]:
def spx_corr(group):
    return group.corrwith(group["SPX"]).drop(["SPX"])

In [156]:
spx_corr(returns)

AAPL    0.564474
MSFT    0.714763
XOM     0.764643
dtype: float64

In [157]:
for item in returns.iterrows():
    print(item.year)

AttributeError: 'tuple' object has no attribute 'year'

In [158]:
def get_year(x):
    return x.year

In [215]:
by_year = returns.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM
2003,0.541124,0.745174,0.661265
2004,0.374283,0.588531,0.557742
2005,0.467540,0.562374,0.631010
2006,0.428267,0.406126,0.518514
2007,0.508118,0.658770,0.786264
2008,0.681434,0.804626,0.828303
2009,0.707103,0.654902,0.797921
2010,0.710105,0.730118,0.839057
2011,0.691931,0.800996,0.859975


example: linear regression

In [195]:
import statsmodels.api as sm
def regress(data, yvar=None, xvar=None):
    Y = data[yvar]
    X = data[[xvar]]
    print(Y.shape, X.shape)
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [196]:
by_year.apply(regress, yvar="AAPL", xvar="SPX")

(251,) (251, 1)
(252,) (252, 1)
(252,) (252, 1)
(251,) (251, 1)
(251,) (251, 1)
(253,) (253, 1)
(252,) (252, 1)
(252,) (252, 1)
(199,) (199, 1)


,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


# Group Transforms & Unwrapped GroupBys
- transform restricts the group from being mutated
- it can broadcast a value to shape of group
- produce an object same shape as group as apply

In [220]:
by_year.transform("mean")

,AAPL,MSFT,XOM,SPX
2003-01-03,0.001736,0.000275,0.000748,0.000859
2003-01-06,0.001736,0.000275,0.000748,0.000859
2003-01-07,0.001736,0.000275,0.000748,0.000859
2003-01-08,0.001736,0.000275,0.000748,0.000859
2003-01-09,0.001736,0.000275,0.000748,0.000859
...,...,...,...,...
2011-10-10,0.001487,0.000090,0.000548,-0.000033
2011-10-11,0.001487,0.000090,0.000548,-0.000033
2011-10-12,0.001487,0.000090,0.000548,-0.000033
2011-10-13,0.001487,0.000090,0.000548,-0.000033


In [225]:
# compare with apply
returns_mean = by_year.apply("mean")

In [226]:
returns_std = by_year.std()

In [227]:
def normalize(group):
    return (group - group.mean()) / group.std()

by_year.transform(normalize)

,AAPL,MSFT,XOM,SPX
2003-01-03,0.214839,0.063136,-0.005756,-0.126933
2003-01-06,-0.074310,0.974927,2.167394,2.043482
2003-01-07,-0.189193,1.034228,-3.128141,-0.699879
2003-01-08,-0.938252,-1.572317,-0.444169,-1.412779
2003-01-09,0.278388,1.587346,1.852824,1.751526
...,...,...,...,...
2011-10-10,3.023523,1.749931,2.274107,2.411777
2011-10-11,1.698285,0.142789,-0.042388,0.040764
2011-10-12,0.197444,-0.104954,0.694252,0.693922
2011-10-13,0.849671,0.539125,-0.673353,-0.207605


In [238]:
(returns.loc["2003-01-06", "AAPL"] - returns_mean.loc[2003, "AAPL"])/returns_std.loc[2003, "AAPL"]

-0.07430950480724965

In [239]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df                   


,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [240]:
def groupbytwo(group):
    return group * 2
df.groupby("key").transform(groupbytwo)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


In [241]:
def group_rank(group):
    return group.rank(ascending=False)
df.groupby("key").transform(group_rank)

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


In [243]:
df.groupby("key").count()

,value
key,
a,4
b,4
c,4


# Pivot Tables
- made possible using `groupby` and reshape
- allows for grouping in row as well column direction simultaneously
- aggregate tabular data using one or more keys and arrange using group keys along rows and other keys/features along columns
- `pivot_table` function in pandas

In [200]:
tips = pd.read_csv("examples/tips.csv")
tips["tips_pct"] = tips["tip"]/tips["total_bill"]

In [201]:
tips.groupby(["day", "smoker"]).agg("mean")

total_bill       tip      size  tips_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [202]:
tips.pivot_table(index=["day", "smoker"])

size       tip  tips_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [203]:
tips.groupby(["day", "time"])[["tips_pct", "size"]].mean()

tips_pct      size
day  time                      
Fri  Dinner  0.158916  2.166667
     Lunch   0.188765  2.000000
Sat  Dinner  0.153152  2.517241
Sun  Dinner  0.166897  2.842105
Thur Dinner  0.159744  2.000000
     Lunch   0.161301  2.459016

row and column grouping in one go

In [205]:
tips.pivot_table(index=["day", "time"], columns="smoker", values=["tips_pct", "size"])

size            tips_pct          
smoker             No       Yes        No       Yes
day  time                                          
Fri  Dinner  2.000000  2.222222  0.139622  0.165347
     Lunch   3.000000  1.833333  0.187735  0.188937
Sat  Dinner  2.555556  2.476190  0.158048  0.147906
Sun  Dinner  2.929825  2.578947  0.160113  0.187250
Thur Dinner  2.000000       NaN  0.159744       NaN
     Lunch   2.500000  2.352941  0.160311  0.163863

In [207]:
tips.pivot_table(index=["day", "time"], columns="smoker", values=["tips_pct", "size"],
    margins=True)


size                      tips_pct                    
smoker             No       Yes       All        No       Yes       All
day  time                                                              
Fri  Dinner  2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
     Lunch   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
Sat  Dinner  2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
Sun  Dinner  2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
Thur Dinner  2.000000       NaN  2.000000  0.159744       NaN  0.159744
     Lunch   2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [208]:

tips.pivot_table(index=["day", "time"], columns="smoker", values=["tips_pct", "size"],
    aggfunc=len,
    margins=True)

size            tips_pct           
smoker          No   Yes  All       No   Yes  All
day  time                                        
Fri  Dinner    3.0   9.0   12      3.0   9.0   12
     Lunch     1.0   6.0    7      1.0   6.0    7
Sat  Dinner   45.0  42.0   87     45.0  42.0   87
Sun  Dinner   57.0  19.0   76     57.0  19.0   76
Thur Dinner    1.0   NaN    1      1.0   NaN    1
     Lunch    44.0  17.0   61     44.0  17.0   61
All          151.0  93.0  244    151.0  93.0  244

In [210]:

tips.pivot_table(index=["day", "time"], columns="smoker", values=["tips_pct", "size"],
    aggfunc="count",
    margins=True)

size            tips_pct           
smoker          No   Yes  All       No   Yes  All
day  time                                        
Fri  Dinner    3.0   9.0   12      3.0   9.0   12
     Lunch     1.0   6.0    7      1.0   6.0    7
Sat  Dinner   45.0  42.0   87     45.0  42.0   87
Sun  Dinner   57.0  19.0   76     57.0  19.0   76
Thur Dinner    1.0   NaN    1      1.0   NaN    1
     Lunch    44.0  17.0   61     44.0  17.0   61
All          151.0  93.0  244    151.0  93.0  244

In [212]:

tips.pivot_table(index=["day", "time"], columns="smoker", values=["tips_pct", "size"],
    aggfunc="count",
    fill_value=0,
    margins=True)

size          tips_pct         
smoker        No Yes  All       No Yes  All
day  time                                  
Fri  Dinner    3   9   12        3   9   12
     Lunch     1   6    7        1   6    7
Sat  Dinner   45  42   87       45  42   87
Sun  Dinner   57  19   76       57  19   76
Thur Dinner    1   0    1        1   0    1
     Lunch    44  17   61       44  17   61
All          151  93  244      151  93  244

# Cross Tabulations
- special case of pivot_table which computes group frequencies

In [213]:
pd.crosstab(index=[tips["day"], tips["time"]], columns=tips["smoker"], margins=True)

smoker        No  Yes  All
day  time                 
Fri  Dinner    3    9   12
     Lunch     1    6    7
Sat  Dinner   45   42   87
Sun  Dinner   57   19   76
Thur Dinner    1    0    1
     Lunch    44   17   61
All          151   93  244